
# 🤖 MGMT 467 - Unit 2 Lab 2: Prompt Studio for AI-Assisted SQL + ML

**Date:** 2025-10-16  
**Objective:** Build and refine a complete ML pipeline for churn prediction using BigQuery — but with **Gemini-style prompts** guiding SQL generation.

You'll learn to:
- Frame SQL goals as clear prompts
- Generate, test, and debug queries with an AI assistant
- Reflect on each modeling step and your prompt design



## Task 0: Connect to BigQuery

**🎯 Goal:** Verify BigQuery access from Colab.  
**📌 Requirements:** Use `%%bigquery`, get current date and user session.

---

### 🧠 Prompt Template  
> Write a SQL query that returns CURRENT_DATE() and SESSION_USER(). I will run it with %%bigquery in Colab.

---

### 👩‍🏫 Example Prompt  
> Write a SQL query using BigQuery syntax that returns today’s date and the current session user.

---

### ✅ Expected SQL Output
```sql
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;
```

---

### 🔍 Checkpoint  
Query should return a single row with today's date and your user.


In [7]:
%%bigquery --project my-project-mgmt-467
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;

Query is running:   0%|          |

Downloading:   0%|          |

,today,user
0,2025-10-26,joyzhang6303@gmail.com



## Task 1: Prepare ML Table

**🎯 Goal:** Create a clean features table for modeling churn.  
**📌 Requirements:** Use cleaned_features as source, select relevant columns, filter rows with churn_label IS NOT NULL.

---

### 🧠 Prompt Template  
> Write a query that creates a new table with columns: [region, plan_tier, age_band, ...] and churn_label from [source_table]. Filter to rows where churn_label IS NOT NULL.

---

### 👩‍🏫 Example Prompt  
> Create a BigQuery table named churn_features from cleaned_features with selected features and where churn_label IS NOT NULL.

---

### ✅ Expected SQL Output
```sql
CREATE OR REPLACE TABLE `your_dataset.churn_features` AS
SELECT region, plan_tier, age_band, avg_rating, total_minutes, churn_label
FROM `your_dataset.cleaned_features`
WHERE churn_label IS NOT NULL;
```

---

### 🔍 Checkpoint  
Table should appear in BigQuery and contain non-null labels.


In [13]:
%%bigquery --project my-project-mgmt-467
SELECT table_name, column_name, data_type
FROM `netflix.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name IN ('movies', 'recommendation_logs', 'reviews', 'search_logs', 'users', 'watch_history')
ORDER BY table_name, column_name;

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,column_name,data_type
0,movies,added_to_platform,DATE
1,movies,box_office_revenue,FLOAT64
2,movies,content_type,STRING
3,movies,content_warning,BOOL
4,movies,country_of_origin,STRING
...,...,...,...
75,watch_history,session_id,STRING
76,watch_history,user_id,STRING
77,watch_history,user_rating,INT64
78,watch_history,watch_date,DATE


In [32]:
%%bigquery --project my-project-mgmt-467
CREATE OR REPLACE TABLE `my-project-mgmt-467.netflix.churn_features` AS
WITH last_watch AS (
  SELECT
    user_id,
    MAX(DATE(watch_date))              AS last_watch_date,
    COUNT(*)                           AS watch_events,
    COUNT(DISTINCT DATE(watch_date))   AS active_watch_days
  FROM `my-project-mgmt-467.netflix.watch_history`
  WHERE user_id IS NOT NULL AND watch_date IS NOT NULL
  GROUP BY user_id
)
SELECT
  user_id,
  watch_events,
  active_watch_days,
  DATE_DIFF(CURRENT_DATE(), last_watch_date, DAY) AS days_since_last_activity,
  IF(last_watch_date < DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY), 1, 0) AS churn_label
FROM last_watch;


Query is running:   0%|          |

""


In [33]:
%%bigquery --project my-project-mgmt-467
SELECT *
FROM `my-project-mgmt-467.netflix.churn_features`
LIMIT 20;


Query is running:   0%|          |

Downloading:   0%|          |

,user_id,watch_events,active_watch_days,days_since_last_activity,churn_label
0,user_05355,16,2,-51,0
1,user_06905,16,2,-48,0
2,user_02894,16,2,-47,0
3,user_06400,16,2,-30,0
4,user_09799,16,2,-23,0
5,user_00857,16,2,9,0
6,user_08992,16,2,45,1
7,user_01414,16,2,53,1
8,user_08518,16,2,71,1
9,user_01431,16,2,81,1



## Task 2: Train Logistic Regression Model

**🎯 Goal:** Train a basic BQML logistic regression model.  
**📌 Requirements:** Use churn_features table, predict churn_label from features.

---

### 🧠 Prompt Template  
> Write a CREATE MODEL SQL for logistic regression using churn_label as label and [features] as inputs.

---

### 👩‍🏫 Example Prompt  
> Train a logistic regression model to predict churn_label using region, plan_tier, total_minutes, avg_rating.

---

### ✅ Expected SQL Output
```sql
CREATE OR REPLACE MODEL `your_dataset.churn_model`
OPTIONS(model_type='logistic_reg') AS
SELECT region, plan_tier, total_minutes, avg_rating, churn_label
FROM `your_dataset.churn_features`;
```

---

### 🔍 Checkpoint  
Model appears in BigQuery under Models. Training completes.


In [34]:
%%bigquery --project my-project-mgmt-467
CREATE OR REPLACE MODEL `my-project-mgmt-467.netflix.churn_model`
OPTIONS (
  model_type = 'logistic_reg',
  input_label_cols = ['churn_label'],
  auto_class_weights = TRUE
) AS
SELECT
  watch_events,
  active_watch_days,
  days_since_last_activity,
  churn_label
FROM `my-project-mgmt-467.netflix.churn_features`;


Query is running:   0%|          |

""


In [35]:
%%bigquery --project my-project-mgmt-467
SELECT model_name, model_type, creation_time
FROM `my-project-mgmt-467`.`region-us`.INFORMATION_SCHEMA.MODELS
WHERE model_name = 'churn_model';


Executing query with job ID: 323328e5-d243-4270-b89e-c1b0a877f44a
Query executing: 0.31s


ERROR:
 403 Access Denied: Table my-project-mgmt-467:region-us.INFORMATION_SCHEMA.MODELS: User does not have permission to query table my-project-mgmt-467:region-us.INFORMATION_SCHEMA.MODELS, or perhaps it does not exist.; reason: accessDenied, message: Access Denied: Table my-project-mgmt-467:region-us.INFORMATION_SCHEMA.MODELS: User does not have permission to query table my-project-mgmt-467:region-us.INFORMATION_SCHEMA.MODELS, or perhaps it does not exist.

Location: US
Job ID: 323328e5-d243-4270-b89e-c1b0a877f44a




## Task 3: Evaluate Model

**🎯 Goal:** Evaluate the logistic regression model.  
**📌 Requirements:** Use ML.EVALUATE.

---

### 🧠 Prompt Template  
> Write a query to evaluate my logistic regression model using ML.EVALUATE.

---

### 👩‍🏫 Example Prompt  
> Evaluate the churn_model using ML.EVALUATE to get accuracy, precision, recall.

---

### ✅ Expected SQL Output
```sql
SELECT * FROM ML.EVALUATE(MODEL `your_dataset.churn_model`);
```

---

### 🔍 Checkpoint  
View performance metrics: accuracy, log_loss, precision, recall.


In [36]:
%%bigquery --project my-project-mgmt-467
SELECT *
FROM ML.EVALUATE(MODEL `my-project-mgmt-467.netflix.churn_model`);


Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.925043,1.0,0.978055,0.961062,0.07691,1.0



## Task 4: Predict Churn

**🎯 Goal:** Use ML.PREDICT to generate churn predictions.  
**📌 Requirements:** Apply model to same input table.

---

### 🧠 Prompt Template  
> Generate SQL to use ML.PREDICT on churn_model and return predictions by user_id.

---

### 👩‍🏫 Example Prompt  
> Predict churn using churn_model. Include user_id, predicted_churn_label, and prediction probability.

---

### ✅ Expected SQL Output
```sql
SELECT user_id, predicted_churn_label, predicted_churn_label_probs
FROM ML.PREDICT(MODEL `your_dataset.churn_model`,
      (SELECT * FROM `your_dataset.churn_features`));
```

---

### 🔍 Checkpoint  
Inspect top churn risk users. Validate probabilities.


In [37]:
%%bigquery --project my-project-mgmt-467
SELECT
  user_id,
  predicted_churn_label,
  (SELECT p.prob FROM UNNEST(predicted_churn_label_probs) p WHERE p.label = 1) AS churn_probability
FROM ML.PREDICT(
  MODEL `my-project-mgmt-467.netflix.churn_model`,
  (SELECT * FROM `my-project-mgmt-467.netflix.churn_features`)
)
ORDER BY churn_probability DESC
LIMIT 20;


Query is running:   0%|          |

Downloading:   0%|          |

,user_id,predicted_churn_label,churn_probability
0,user_01200,1,1.0
1,user_04811,1,1.0
2,user_03002,1,1.0
3,user_01600,1,1.0
4,user_07325,1,1.0
5,user_09139,1,1.0
6,user_02324,1,1.0
7,user_04604,1,1.0
8,user_03041,1,1.0
9,user_08285,1,1.0


# Task
Create a BigQuery table named `churn_features` from the `netflix` dataset, selecting relevant columns for churn prediction and filtering for rows where `churn_label` is not null.

## Understand netflix data for churn prediction

### Subtask:
Examine the schema of the Netflix tables provided (movies, recommendation_logs, reviews, search_logs, users, watch_history) to identify columns that could be relevant for predicting user churn.


**Reasoning**:
I need to query the BigQuery INFORMATION_SCHEMA to understand the schema of the tables in the `netflix` dataset to identify relevant columns for churn prediction.



## Identify features for churn features table

### Subtask:
Based on the available columns in the Netflix tables, determine which ones to include in the `churn_features` table. This might involve selecting columns directly or potentially joining tables and aggregating data to create features.


**Reasoning**:
Based on the schema information obtained in the previous step, I will select relevant columns from the `users`, `watch_history`, `reviews`, and `recommendation_logs` tables to create the `churn_features` table. I will include `user_id`, `region`, `plan_tier`, `age_band` from the `users` table. From `watch_history`, I will aggregate `watch_duration_minutes` to get `total_minutes`. From `reviews`, I will aggregate `rating` to get `avg_rating`. From `recommendation_logs`, I will aggregate to get the count of recommendations viewed or clicked, here I will use the count of `recommendation_id` as `total_recommendations`. Finally, I will include `churn_label` from the `users` table and filter for rows where `churn_label` is not null.



**Reasoning**:
The previous query failed because `churn_label` was not found inside the `u` alias (which refers to the `users` table). However, based on the schema information, `churn_label` *is* in the `users` table. The error might be a transient issue or a problem with how the alias was being used in the `WHERE` and `GROUP BY` clauses. I will try the query again, ensuring the `churn_label` column is correctly referenced with the `u` alias.



## Identify features for churn features table

### Subtask:
Based on the available columns in the Netflix tables, determine which ones to include in the `churn_features` table. This might involve selecting columns directly or potentially joining tables and aggregating data to create features.


**Reasoning**:
The previous attempt to create the `churn_features` table failed because the `churn_label` column was not recognized in the users table alias. This is likely an issue with how the alias was used or a potential discrepancy in the schema information. To address this, I will attempt to create the table again, ensuring the alias `u` is correctly used to reference columns from the `users` table, including `churn_label`. I will also explicitly select all the identified relevant columns and perform the necessary joins and aggregations as outlined in the instructions.

